# Setup

## Install & Improt dependencies

In [3]:
!pip install tensorflow opencv-python matplotlib

In [4]:
pip list

Package                      Version
---------------------------- ---------
absl-py                      1.4.0
asttokens                    2.2.1
astunparse                   1.6.3
backcall                     0.2.0
cachetools                   5.3.0
certifi                      2022.12.7
charset-normalizer           3.0.1
colorama                     0.4.6
comm                         0.1.2
contourpy                    1.0.7
cycler                       0.11.0
debugpy                      1.6.6
decorator                    5.1.1
executing                    1.2.0
flatbuffers                  23.3.3
fonttools                    4.38.0
gast                         0.4.0
google-auth                  2.16.2
google-auth-oauthlib         0.4.6
google-pasta                 0.2.0
grpcio                       1.51.3
h5py                         3.8.0
idna                         3.4
ipykernel                    6.21.2
ipython                      8.11.0
jax                          0.4.5
jedi 

In [12]:
import tensorflow as tf
import os

## Remove invalid images

In [6]:
import cv2
import imghdr

C:\Users\krios\AppData\Local\Temp\ipykernel_5868\4232469594.py:2: DeprecationWarning: 'imghdr' is deprecated and slated for removal in Python 3.13
  import imghdr


In [7]:
data_dir = "data"

In [8]:
img_extensions = ["jpeg", "jpg", "bmp", "png"]

In [9]:
deleted_imgs = 0
for img_class in os.listdir(data_dir):
    for img in os.listdir(os.path.join(data_dir, img_class)):
        img_path = os.path.join(data_dir, img_class, img)
        try:
            # Trying to open an image using cv2
            image = cv2.imread(img_path)
            extension = imghdr.what(img_path)
            if extension not in img_extensions:
                print(f"Image not in extension list {img_path}")
                os.remove(img_path)
                deleted_imgs += 1
        except Exception as e:
            print(f"Issue with image {img_path}")
print(f"Overall deleted images = {deleted_imgs}")

Overall deleted images = 0


## Load data

In [10]:
import numpy as np
from matplotlib import pyplot as plt

In [13]:
data = tf.keras.utils.image_dataset_from_directory("data")

Found 245 files belonging to 2 classes.


In [14]:
data_iterator = data.as_numpy_iterator()

In [15]:
batch = data_iterator.next()